# Батарейка дня subprocess
Запуск процессов ОС и коммуникация с ними

Примеры работают на python-3.5

см [докуменацию](https://docs.python.org/3.5/library/subprocess.html?highlight=subprocess#module-subprocess)

`pydoc3 subprocess`

    Security
    --------
    Unlike some other popen functions, this implementation will never call
    /bin/sh implicitly.  This means that all characters, including shell
    metacharacters, can safely be passed to child processes.

### Как еще можно запускать программы?
```
import os
assert os.system('whoami') == 0

pid = os.spawnlp(os.P_NOWAIT, "/bin/mycmd", "mycmd", "myarg")
```

In [58]:
import subprocess
print(subprocess.Popen.__doc__)
subprocess.Popen('true')

None


In [59]:
sub = _
assert sub.returncode is None
pid = sub.pid
pid

7897

In [60]:
print('ps x -p %d' % pid)

ps x -p 7897


In [21]:
code = sub.poll()  # try now
assert sub.returncode == code
code

0

In [27]:
sub = subprocess.Popen('sleep 1')

FileNotFoundError: [Errno 2] No such file or directory: 'sleep 1'

In [47]:
sub = subprocess.Popen('sleep 1', shell=True)
sub.pid

7879

In [40]:
import time

start = time.time()
sub = subprocess.Popen(['sleep', '4'])

print('Checking for a child %d' % sub.pid)
assert sub.poll() is None

for i in range(4):
    print('Waiting a bit')
    time.sleep(0.5)
    assert sub.poll() is None

print('Waiting for a child %d' % sub.pid)
sub.wait()

print('Took %1.3f' % (time.time() - start))

Checking for a child 7860
Waiting a bit
Waiting a bit
Waiting a bit
Waiting a bit
Waiting for a child 7860
Took 4.008


# Мне нужен результат этой работы

In [22]:
sub = subprocess.Popen('date')
assert sub.stdout is None

In [24]:
print(sub.communicate.__doc__)
sub.communicate()

Interact with process: Send data to stdin.  Read data from
        stdout and stderr, until end-of-file is reached.  Wait for
        process to terminate.

        The optional "input" argument should be data to be sent to the
        child process (if self.universal_newlines is True, this should
        be a string; if it is False, "input" should be bytes), or
        None, if no data should be sent to the child.

        communicate() returns a tuple (stdout, stderr).  These will be
        bytes or, if self.universal_newlines was True, a string.
        


(None, None)

In [68]:
sub = subprocess.Popen('date', stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
out, err = sub.communicate()
assert err is None
out

b'\xd0\xb2\xd1\x82\xd0\xbe\xd1\x80\xd0\xbd\xd0\xb8\xd0\xba,  9 \xd0\xbc\xd0\xb0\xd1\x8f 2017 \xd0\xb3. 21:15:12 (MSK)\n'

In [69]:
out.decode('utf')

'вторник,  9 мая 2017 г. 21:15:12 (MSK)\n'

In [66]:
sub = subprocess.Popen('which', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
out, err = sub.communicate()
assert out == b''
err

b'usage: which [-as] program ...\n'

In [67]:
sub = subprocess.Popen('which', stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
out, err = sub.communicate()
assert err is None
out

b'usage: which [-as] program ...\n'

In [76]:
cmd = 'ps ax -U nesusvet'.split()
sub = subprocess.Popen(cmd, stdout=subprocess.PIPE)
out, err = sub.communicate()
out.split(b'\n')

[b'  PID   TT  STAT      TIME COMMAND',
 b'    1   ??  Ss     5:43.76 /sbin/launchd',
 b'   43   ??  Ss     2:06.59 /usr/sbin/syslogd',
 b'   44   ??  Ss     0:41.36 /usr/libexec/UserEventAgent (System)',
 b'   46   ??  Ss     1:46.06 /usr/libexec/kextd',
 b'   47   ??  Ss     1:07.55 /System/Library/Frameworks/CoreServices.framework/Versions/A/Frameworks/FSEvents.framework/Versions/A/Support/fseventsd',
 b'   49   ??  Ss     0:20.82 /opt/cisco/anyconnect/bin/vpnagentd -execv_instance',
 b'   52   ??  Ss     0:00.72 /System/Library/CoreServices/appleeventsd --server',
 b'   53   ??  Ss     0:45.47 /usr/libexec/configd',
 b'   54   ??  Ss     0:16.61 /System/Library/CoreServices/powerd.bundle/powerd',
 b'   59   ??  Ss     1:00.61 /usr/libexec/airportd',
 b'   61   ??  SNs    0:00.73 /usr/libexec/warmd',
 b'   62   ??  Ss     2:25.25 /System/Library/Frameworks/CoreServices.framework/Frameworks/Metadata.framework/Support/mds',
 b'   66   ??  Ss     0:00.20 /System/Library/CoreServices/ic

In [77]:
print(subprocess.check_output.__doc__)

Run command with arguments and return its output.

    If the exit code was non-zero it raises a CalledProcessError.  The
    CalledProcessError object will have the return code in the returncode
    attribute and output in the output attribute.

    The arguments are the same as for the Popen constructor.  Example:

    >>> check_output(["ls", "-l", "/dev/null"])
    b'crw-rw-rw- 1 root root 1, 3 Oct 18  2007 /dev/null\n'

    The stdout argument is not allowed as it is used internally.
    To capture standard error in the result, use stderr=STDOUT.

    >>> check_output(["/bin/sh", "-c",
    ...               "ls -l non_existent_file ; exit 0"],
    ...              stderr=STDOUT)
    b'ls: non_existent_file: No such file or directory\n'

    There is an additional optional argument, "input", allowing you to
    pass a string to the subprocess's stdin.  If you use this argument
    you may not also use the Popen constructor's "stdin" argument, as
    it too will be used internally.  

In [ ]:
cmd = 'ps aux'.split()
out = subprocess.check_output(cmd)  # По-проще
out.split(b'\n')

# Аналог ps aux | grep python

In [3]:
ps_aux = subprocess.Popen(
    'ps aux'.split(),
    stdout=subprocess.PIPE,
)
grep = subprocess.Popen(
    'grep python'.split(),
    stdin=ps_aux.stdout,
    stdout=subprocess.PIPE,
)
out, err = grep.communicate()
out.split(b'\n')

[b'nesusvet          7774   0,0  0,0  2423376    268   ??  R     8:25     0:00.00 grep python',
 b'nesusvet          7767   0,0  0,5  2473712  41944   ??  Ss    8:24     0:01.10 /Users/nesusvet/dev/battery-of-the-day/venv/bin/python3.5 -m ipykernel_launcher -f /Users/nesusvet/Library/Jupyter/runtime/kernel-2c57b4a5-37c8-42b6-a059-70592cf66096.json',
 b'nesusvet          7374   0,0  0,3  2465332  29264 s004  S+    5:14     0:06.18 /Users/nesusvet/dev/battery-of-the-day/venv/bin/python3.5 /Users/nesusvet/dev/battery-of-the-day/venv/bin/jupyter-notebook',
 b'']

In [5]:
import os
def show_tree():
    pid = os.getpid()
    cmd = 'pstree', '-p', str(pid)
    text = subprocess.check_output(cmd)
    print(text.decode('utf'))

show_tree()

-+= 00001 root /sbin/launchd
 \-+= 00741 nesusvet tmux
   \-+= 06921 nesusvet -zsh
     \-+= 07374 nesusvet /Users/nesusvet/dev/battery-of-the-day/venv/bin/python3.5 /Users/nesusvet/dev/battery-of-the-day/venv/bin/jupyter-notebook
       \-+= 07767 nesusvet /Users/nesusvet/dev/battery-of-the-day/venv/bin/python3.5 -m ipykernel_launcher -f /Users/nesusvet/Library/Jupyter/runtime/kernel-2c57b4a5-37c8-42b6-a059-70592cf66096.json
         |--- 07772 nesusvet (true)
         |--- 07773 root (ps)
         \-+- 07775 nesusvet pstree -p 7767
           \--- 07776 root ps -axwwo user,pid,ppid,pgid,command



In [ ]:
print(os.kill.__doc__)
print(os.waitpid.__doc__)

In [ ]:
os.wait()

In [ ]:
show_tree()

In [53]:
print(subprocess.call.__doc__)
try:
    print('Run with timeout')
    subprocess.call(['sleep', '5'], timeout=3)
except subprocess.TimeoutExpired as ex:
    print('An exception occured %s' % ex)

Run command with arguments.  Wait for command to complete or
    timeout, then return the returncode attribute.

    The arguments are the same as for the Popen constructor.  Example:

    retcode = call(["ls", "-l"])
    
Run with timeout
An exception occured Command '['sleep', '5']' timed out after 3 seconds


In [54]:
print(subprocess.check_call.__doc__)
subprocess.check_call(['which', 'python'])

Run command with arguments.  Wait for command to complete.  If
    the exit code was zero then return, otherwise raise
    CalledProcessError.  The CalledProcessError object will have the
    return code in the returncode attribute.

    The arguments are the same as for the call function.  Example:

    check_call(["ls", "-l"])
    


0

In [55]:
subprocess.check_call(['which', 'flynn'])

CalledProcessError: Command '['which', 'flynn']' returned non-zero exit status 1

# Python вызывает другой python-процесс :)

In [7]:
text = '''
import time

def main():
    time.sleep(5)

if __name__ == '__main__':
    print('Waiting now...')
    main()
'''
with open('tmp.py', 'w') as file_output:
    file_output.write(text)

In [13]:
factory = (subprocess.Popen('python tmp.py'.split()) for _ in range(10))
for child in factory:
    print('New born child pid = %d' % child.pid)

print('Process tree')
show_tree()

print('Waiting 10 sec')
time.sleep(10)

show_tree()
print('Where are my children?')

New born child pid = 7823
New born child pid = 7824
New born child pid = 7825
New born child pid = 7826
New born child pid = 7827
New born child pid = 7828
New born child pid = 7829
New born child pid = 7830
New born child pid = 7831
New born child pid = 7832
Process tree
-+= 00001 root /sbin/launchd
 \-+= 00741 nesusvet tmux
   \-+= 06921 nesusvet -zsh
     \-+= 07374 nesusvet /Users/nesusvet/dev/battery-of-the-day/venv/bin/python3.5 /Users/nesusvet/dev/battery-of-the-day/venv/bin/jupyter-notebook
       \-+= 07767 nesusvet /Users/nesusvet/dev/battery-of-the-day/venv/bin/python3.5 -m ipykernel_launcher -f /Users/nesusvet/Library/Jupyter/runtime/kernel-2c57b4a5-37c8-42b6-a059-70592cf66096.json
         |--- 07772 nesusvet (true)
         |--- 07773 root (ps)
         |--- 07823 nesusvet python tmp.py
         |--- 07824 nesusvet python tmp.py
         |--- 07825 nesusvet python tmp.py
         |--- 07826 nesusvet python tmp.py
         |--- 07827 nesusvet python tmp.py
         |--- 07

In [14]:
os.wait()

(7832, 0)

In [56]:
show_tree()

-+= 00001 root /sbin/launchd
 \-+= 00741 nesusvet tmux
   \-+= 06921 nesusvet -zsh
     \-+= 07374 nesusvet /Users/nesusvet/dev/battery-of-the-day/venv/bin/python3.5 /Users/nesusvet/dev/battery-of-the-day/venv/bin/jupyter-notebook
       \-+= 07767 nesusvet /Users/nesusvet/dev/battery-of-the-day/venv/bin/python3.5 -m ipykernel_launcher -f /Users/nesusvet/Library/Jupyter/runtime/kernel-2c57b4a5-37c8-42b6-a059-70592cf66096.json
         |--- 07772 nesusvet (true)
         |--- 07773 root (ps)
         |--- 07861 nesusvet (true)
         |--- 07879 nesusvet (sleep)
         \-+- 07888 nesusvet pstree -p 7767
           \--- 07889 root ps -axwwo user,pid,ppid,pgid,command



# Применение
Я, как разработчик проекта, хочу помнить и делать как можно меньше ручных действий. Например, при изменении требований к параметрам запросов к API и изменении формата ответа, нужно регулярно обновлять документацию, что сводится к вызову `make`-задачи. Почему бы не использовать механизм хуков, существующий в `git`?

In [ ]:
#!/usr/bin/env python

import re
import subprocess

RE_RAW_DOC = re.compile(r'\.(json|raml)$')
RE_COMPILED_DOC = re.compile(r'\.(md)$')

GIT_LS_FILES = 'git diff --name-only --cached'.split()
GIT_ADD = 'git add -u'.split()

REBUILD_DOCS = 'make generate_docs'.split()


def iter_modified():
    output = subprocess.check_output(GIT_LS_FILES).decode('utf')
    for line in output.split('\n'):
        yield line


def docs_need_rebuild():
    for filename in iter_modified():
        if RE_RAW_DOC.search(filename):
            yield filename


def docs_was_rebuilt():
    for filename in iter_modified():
        if RE_COMPILED_DOC.search(filename):
            yield filename


def main():
    modified = list(docs_need_rebuild())
    rebuilt_docs = list(docs_was_rebuilt())
    if modified and not rebuilt_docs:
        subprocess.check_call(REBUILD_DOCS)

        rebuilt_docs = list(docs_was_rebuilt())
        subprocess.check_call(GIT_ADD + rebuilt_docs)


if __name__ == '__main__':
    main()

# Напоследок
### python-3.5
`subprocess.run() => subprocess.CompletedProcess`

# Дальше и глубже
- pipes
- multiprocessing